---
# **Modelling Spatial Representations** [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/TomGeorge1234/ModellingSpatialRepresentations/blob/master/modelling_spatial_representations.ipynb)
### **TReND CaMinA, Computational Neuroscience and Machine Learning Summer School, 2024**
#### made by: **Tom George, UCL**

---

Ιn this tutorial we write code to model spatial behaviour and spatial representations in a variety of environments. To do so we'll use the `RatInABox` simulation toolkit (full documentation can be found [here](https://github.com/TomGeorge1234/RatInABox)).

In summary, the goal of this package is to make it easy to generate spatial behaviour and spatial representation neuronal data in complex worlds. 

Researchers use it to: 
* Generate data to test theories.
* Build models of behaviour and Hippocampal networks.
* Compare or augment neural data with theoretical models. 

<center><img src="./figures/display_figures/ratinabox.gif" width=1000></center>


---

## **Learning Objectives**
* Install and Import the `RatInABox` package for modelling spatial behaviour and representations.
* Learn its basic API: be able to build  2D Environments and generate locomotion trajectories for one or multiple Agents _within_ these Environments.
* Learn how to generate single cell representations: place cells, grid cells, head direction cells and boundary cells
* Built a simple feed-forward threshold model for how grid cells could drive place cells.   

---

## **Contents** 
0. [Import dependencies](#dependencies)
1. [Generating trajectories in 2D Environments](#trajectories)
    1. [Square environment and random trajectories](#square)
    2. [Building complex environments](#complex-environments)
    3. [Varying the trajectory parameters](#changing-trajectories)
    4. [Controlling the Agent for non-random trajectories](#non-random)
    5. [Animating trajectories](#animating)
    6. [Multiple Agents](#multiagent)
    7. [Simulating the behaviour of TReND students](#trend)
2. [Spatial representations](#spatial-reps)
    1. [Place cells](#placecells)
    3. [Grid cells](#gridcells)
    4. [Head direction cells](#headdirectioncells)
    5. [Boundary vector cells](#boundaryvectorcells)
3. [Grid cells drive place cells: A model](#gc2pc)
4. [Advanced exercises](#advanced)


---

## **0 Install and import `RatInABox`** <a name="dependencies"></a>

Below we install and import `RatInABox`. 

Note the line where we set the `ratinabox.figure_directory = './figures/'`. This is where any figures we make will be automatically saved.

In [ ]:
#@title Run this cell to install ratinabox and import the necessary libraries {display-mode: "form" }
!pip install ratinabox>=1.12.4 pandas

import ratinabox
from ratinabox.Environment import Environment
from ratinabox.Agent import Agent 
from ratinabox.Neurons import *
ratinabox.autosave_plots = True; ratinabox.stylize_plots(); ratinabox.figure_directory = './figures/'
import pandas as pd 

In [ ]:
dt = 0.1 # throughout this tutorial we will use a time step of 0.1 seconds

---

## **1.0 Generating random trajectories in 2D Environments** <a name="trajectories"></a>

RatInABox consists of three core classes: 

* `Environment()`: a 1D or 2D arena where the actions take place. 
* `Agent()`: These live inside the `Environment` and move around.
* `Neurons()`: Belong to an `Agent` and report its "state" (e.g. place cells or grid cells)

<center><img src="./figures/display_figures/ratinabox_structure.png" width=400></center>

For now we'll focus on the first two: 

### **1.1 Basic square environment with random trajectory** <a name="square"></a>

If initialised with no parameters then, by default
* The `Environment` will be square
* The `Agent` will move under a smooth random motion policy 

In [ ]:
Env = Environment(params={})
Ag = Agent(Env, params={})

We can plot the environment to see its shape (it's a square!) 

In [ ]:
Env.plot_environment()

You may notice this figure was automatically saved so you can find it later if you want. 

`Agent.update()` progresses the simulation by a small amount of time `Agent.dt`. So for 60 seconds of simulation...

In [ ]:
for i in range(int(60/dt)):
    Ag.update(dt=dt)

Data is saved into a dictionary `Agent.history`

In [ ]:
agent_data = pd.DataFrame(Ag.history)
agent_data

**_📝TASK_** Starting from this `pandas` dataframe calculate the average speed of the `Agent`.

$$speed = \left< \| velocity\| \right>$$

Hint: you might like to loop over each element in the dataframe: `for vel in dataframe['vel']: ...` 

In [ ]:
#@title Double click for the answer {display-mode: "form" }
speeds = []
for vel in agent_data['vel']:
    speed = np.linalg.norm(vel)
    speeds.append(speed)
average_speed = np.mean(speeds)
print(f"The average speed is: {average_speed : .3f} m/s")

#### 1.1.1 Use `Agent.plot_trajectory()` to visualise the path the Agent took 
Thankfully `RatInABox` has in-built plotting functions which means we can quickly visualise this data with relatively little code. 

The most useful one for the `Agent` class is `Agent.plot_trajectory()`

In [ ]:
Ag.plot_trajectory() # plot the trajectory of the agent

### **1.2 Building more _complex_ 2D environments** <a name="complex-environments"></a>

**_📝TASK_**: Use `Environment.add_wall([[x0,y0],[x1,y1])` to add a single wall to the `Environment`

In [ ]:
# your code goes here
Env = Environment()
Env.add_wall([[0.3,0.6],[0.3,0]]) #<-- add you wall in here
Env.plot_environment()

Notice we left the `params` dictionaries empty giving us a default square `Environment`. Here are the default values: 

In [ ]:
Env.get_all_default_params()

**_📝TASK_**: By passing the `boundary` param into a new `Environment` make a non-square `Environment`

The `boundary` parameter should be a list of corners definin the non-square `Environment`
Ideas: 
* Make a T-Maze
* Octagon `Environment`

In [ ]:
Env = Environment(
    params={
        'boundary': [[0.25,0],[0,0.25],[0,0.75],[0.25,1],[0.75,1],[1,0.75],[1,0.25],[0.75,0]]#You list goes here
        }
)
Env.plot_environment()

**_📝TASK_**: Pass the `holes` parameter (this is a list of lists, each inner list element giving the corners of a "hole") to make an `Environment` with a hole

In [ ]:
# your code goes here 
Env = Environment(params={'holes':[]}) #<-- add one or more holes into this list 
Env.plot_environment()

In [ ]:
#@title Double click for the answer {display-mode: "form" }
# your code goes here 
Env = Environment(params={'holes':[[[0.2,0.2],[0.4,0.2],[0.4,0.4],[0.2,0.4]]]}) #<-- add one or more holes into this list 
Env.plot_environment()

## **1.3 Varying the trajectory parameters** <a name="changing-trajectories"></a>

Notice we left the `params` dictionaries empty giving us a default `Agent`. Here are the default values: 

In [ ]:
Agent.get_all_default_params()

For example, the `Agent` parameter `rotational_velocity_std` controls how curvy the trajectory is. 

**_📝TASK_**: By passing the `rotational_velocity_std` (or others) param into a new `Agent` vary the trajectory statistics.

In [ ]:
Env = Environment()
Ag = Agent(Env,params={}) #<--- add parameters here
for i in range(int(60/dt)):
    Ag.update(dt=dt)
Ag.plot_trajectory()

## **1.4 Non-random trajectories** <a name="non-random"></a>

Finally, it's possible to "guide" the `Agent` trajectory, rather than just having a random motion.

To do this we set pass a `drift_velocity` to the Agent on each update set: `Agent.update(drift_velocity=[vx,vy])`

In this example we'll set a drift velocity so that an `Agent` in a T-Maze moves up the central arm then either left or right. Upon reaching the end of the arm it teleports back to the bottom of the central arm. 

In [ ]:
Env = Environment(
    params={
        'boundary': 
        [[-0.1,0],
         [0.1,0],
         [0.1,0.8],
         [0.7,0.8],
         [0.7,1.0],
         [-0.7,1.0],
         [-0.7,0.8],
         [-0.1,0.8]]})
Ag = Agent(Env)
Env.plot_environment()

In [ ]:
def drift_velocity_from_position(pos : np.ndarray):
    """Returns a drift velocity that will move the agent up the central arm and then along the T-arms"""
    if pos[1] < 0.8: return 0.2*np.array([0,1])
    else: 
        if pos[0] < 0: return 0.2*np.array([-1,0])
        elif pos[0] >= 0: return 0.2*np.array([1,0])

In [ ]:
for i in range(int(100/dt)):
    Ag.update(dt=dt,drift_velocity=drift_velocity_from_position(Ag.pos)) # drift along the arms
    if np.abs(Ag.pos[0]) > 0.65: # teleport back and reset the velocity to zero
        Ag.pos = np.array([0,0.05])
        Ag.velocity = np.array([0,0])

Ag.plot_trajectory(color='changing')

**_📝TASK_**: Make a default square environment and set a drift velocity in such a way that the Agent moves around in circles.

In [ ]:
# your code goes here 

In [ ]:
#@title Double click for the answer {display-mode: "form" }
Env = Environment()
Ag = Agent(Env)
for i in range(int(100/dt)):
    [x,y] = Ag.pos - np.array([0.5,0.5]) #distance from the centre of the Environment
    [vx,vy] = [-0.3*y,0.3*x]
    Ag.update(dt=dt, drift_velocity=np.array([vx,vy]))
Ag.plot_trajectory()

## **1.5 Animating trajectories** <a name="animating"></a>
Finally, we can animate the trajectory to see the `Agent` moving around the `Environment` in real time. 
Note: this works best if running locally and can be a little slow on Google colab. Feel free to skip this! 

To do this use the `Agent.animate_trajectory()` function. I recommend passing some parameters: 
* `speed_up`: How many times real speed the animation will play
* `t_end`: when, in Agent time coordinates, to plot until
* `fps`: frame per second of resulting animation (set lower for faster run times)
* `color`: The color of  the trajectory (any `matplotlib` color). If its passed as `"changing"` it will shade the trajectory.
* `autosave`: Whether the animation is automatically saved to the figure directory (recommend `False` to save time) 


In [ ]:
Ag.animate_trajectory(speed_up=10,t_end=60,fps=15,color="changing",autosave=False)

**_📝TASK_**: Go back up to the T-Maze example we built and animate the trajectory of the `Agent`. 

## **1.6 Multiple agents** <a name="multiagent"></a>
Finally we can add multiple Agents to the environment, no problem. 
When ploting, specify `plot_all_agents=True` to see them.

In [ ]:
Env = Environment()
Ag1 = Agent(Env)
Ag2 = Agent(Env)
Ag3 = Agent(Env)
for i in range(int(60/dt)):
    Ag1.update(dt=dt)
    Ag2.update(dt=dt)
    Ag3.update(dt=dt)

Ag1.plot_trajectory(plot_all_agents=True)

**_📝OPTIONAL TASK_**: Make an Environment with multiple `Agents` who "interact".
 
Consider how we could set a `drift_velocity` for each Agent which might make them approach or run away from each another. If you feel comfortable, give it a go! 

In [ ]:
#@title Double click to see my solution to this problem (there might be many ways of doing it) 

# here the algorithm I use is as follows: 
# for each agent, loop over all the _other_ agents and set a drift velocity which points towards them 
# you drift more strongly the closer you are to them

Env = Environment()
Ag1 = Agent(Env)
Ag2 = Agent(Env)
Ag3 = Agent(Env)
Ag4 = Agent(Env)

All_Agents = [Ag1,Ag2,Ag3,Ag4]
for _ in range(int(120/dt)):
    for i in range(len(All_Agents)):
        drift_vel = np.array([0.0,0.0])
        Me = All_Agents[i]
        other_Ags = All_Agents.copy(); other_Ags.pop(i) #remove yourself from the list of other agents
        for other_Ag in other_Ags:
            vec_to_other_agent = np.array(other_Ag.pos) - np.array(Me.pos)
            dist_to_other_agent = np.linalg.norm(vec_to_other_agent)
            norm_vec = vec_to_other_agent / dist_to_other_agent
            attract_speed = 0.005*np.exp(-(dist_to_other_agent/0.2)**2/2)
            drift_vel += attract_speed*norm_vec
        Me.update(dt=dt,drift_velocity=drift_vel)

Ag1.animate_trajectory(speed_up=10,fps=10,plot_all_agents=True,autosave=False)

## **1.7 Simulating the behaviour of TReND students...** <a name="trend"></a>
### **...when they hear about an awesome new summer school happening in Accra**

Let's piece this all together and make one simulation showing the great migration of students from all over Africa to Kigali for the TReND summer school 

We'll go through it step-by-step.

In [ ]:
# Step 1 get the coordinates of the boundary of Africa (
# These are stored as a massive list in the very last cell of this notebook
# Go and run that cell and then come back here

kigali = np.array([0.4,-0.3]) # coordinates of Kigali

# Step 2: create an Environment object with the boundary of Africa as the boundary
Africa = Environment(params={
    'dx': 0.05,
    'boundary': africa_boundary
})


# Step 3: create many Agents (or students) within the Environment you just created
# put them all in a list called Students
Ag0 = Agent(Africa)
Ag1 = Agent(Africa)
Ag2 = Agent(Africa)
Ag3 = Agent(Africa)
Ag4 = Agent(Africa)
Ag5 = Agent(Africa)
Ag6 = Agent(Africa)
Ag7 = Agent(Africa)
Ag8 = Agent(Africa)
Students = [Ag0,Ag1,Ag2,Ag3,Ag4,Ag5,Ag6,Ag7,Ag8]


# Step 4: Simulate!
for i in range(int(60/dt)):
    for Student in Students:
        if Student.t < 30: # initially students just randomly drift around Africa
            Student.update(dt=dt)
        else: # after 30 seconds students drift towards kigali
            vector_to_kigali = kigali - Student.pos
            vector_to_kigali = vector_to_kigali / np.linalg.norm(vector_to_kigali)
            drift_velocity = 0.1 * vector_to_kigali
            Student.update(dt=dt,drift_velocity=drift_velocity)

# Step 5: Plot the trajectories of all the students
Students[0].plot_trajectory(plot_all_agents=True)


# or animate but...this may be VERY slow to run on Google Colab.
# Students[0].animate_trajectory(speed_up=5,fps=10,plot_all_agents=True,autosave=False)

---

## **2.0 Spatial Representations** <a name="spatial-reps"></a>

Great! So far we've
* Successfully been able to make `Environment`s
* We then added `Agent`s to these environments
* We simulate these `Agent`s moving around under complex, realistic motion policies.

But what about what goes on inside the brain? We're neuroscientists after all!

Now we'll add `Neurons` to `Agents`. Whereas `Agents` have continuously updating _positions_ and _velocities_ in an Environment, `Neurons` have continuously updating _firing rates_. 

As we learnt during the lecture, certain types of neurons e.g. place cells or grid cells, have localised receptive field (they only fire in specific regions of the environment), others e.g. head direction cells, are specific to aspects of how the agent is moving. 

Here are some example `Neurons` subclasses we can add to `Agents`

* `PlaceCells`
* `GridCells`
* `BoundaryVectorCells`
* `HeadDirectionCells`

Just like `Agent`s, each is initialised with a dictionary of `params` and can be updated by calling `Neuron.update()`. 

### **2.1 Place cells** <a name="placecells"></a>

Here we'll make a `PlaceCell` which has a simple `gaussian` receptive field: 

$$ f_i(\vec{x}) = \textrm{fr}_{\textrm{max}}\exp \big( -\frac{(\vec{x}(t) - \vec{x}_{i})^2}{2 \sigma^2} \big)$$ 

$\vec{x}_i$ is the centre of the Gauassian and $\sigma$ is the width. 

Let's see how we'd go about making this.

In [ ]:
Env = Environment()
Ag = Agent(Env)
PCs = PlaceCells(Ag, params={'n':5, #<--- 5 place cells
                             'max_fr':5, #<--- max firing rate = 5Hz
                             'widths':0.1, #<--- sigma = 0.1 m
                                     }) 

Note we didn't specify the centre of the place fields, $x_i$, so they are chosen randomly. 

Now lets simulate 10 mins seconds of exploration.

In [ ]:
for i in range(int(10*60/dt)):
    Ag.update(dt=dt)
    PCs.update() # this line updates the firing rate of the place cells

All the data is saved into a dictionary, `Neurons.history`

In [ ]:
import pandas as pd 
neuron_data = pd.DataFrame(PCs.history)
neuron_data

**_📝TASK_**: Calculate the average firing rate of one of the neurons

In [ ]:
# your code goes here

In [ ]:
#@title Double click to see solution {display-mode: "form" }

# To calculate the firing rate we can just see how many spikes we observed in the 10 minutes and divide by 10 minutes
# we'll do this for the first neuron only. 
spikes_bool = [spike_list[0] for spike_list in neuron_data['spikes']]
num_spikes = np.sum(spikes_bool)
av_firing_rate = num_spikes / (10*60)

print(f"Total number of spikes: {num_spikes:g}\nAverage firing rate (spikes / second): {av_firing_rate:.3f}")

`RatInABox` has in-built plotting functions so we can explore this data easily and without much code. These include: 

| Function | Description | 
| --- | --- |
| `Neurons.plot_rate_timeseries()` | show the firing rate of the neurons over time |
| `Neurons.plot_rate_map()` | shows the ground truth firing rate of the neurons as a function of the position in the Environment |
| `Neurons.plot_rate_map(method='neither',spikes=True)` | shows where the neurons spiked, from the data already collected |



In [ ]:
fig, ax = Ag.plot_trajectory()

In [ ]:
fig, ax = PCs.plot_rate_timeseries(t_end=60)
fig.suptitle("The observed spiking rate of the neuron over time ")

In [ ]:
fig, ax = PCs.plot_rate_map()
fig.suptitle("This is the ground-truth receptive field of the place cell (gaussian)")

In [ ]:
fig, ax = PCs.plot_rate_map(method='neither',spikes=True)
fig.suptitle("The is the data (spikes) after 10 minutes of simulation")

To animate this we can call `Agent.animate_trajectory` but passing an additional function which displays the location of the spikes as they appear. 

I've written this function below. Expect this to take about a minute to run, feel free to skip.

In [ ]:
#you can ignore this function for now
def plot_spikes_on_animation(fig, ax, t, **kwargs):
    kwargs['cells'].plot_rate_map(fig=fig,ax=ax,method='neither',spikes=True,t_end=t,autosave=False,chosen_neurons=kwargs['chosen_neurons'])
    return fig, ax

Ag.animate_trajectory(speed_up=30,
                      fps=10,
                      additional_plot_func=plot_spikes_on_animation,
                      cells=PCs,
                      chosen_neurons='1',
                      autosave=False)

**_📝TASK_**: Start playing around with these cells, what do you see?

Good ideas include

* Number: Try simulating more than place cells, e.g., 10, using `params['n']=10`
* Widths: Vary the widths of the place cells using `params['widths']`
* Walls: Add a single wall to the Environment using `Env.add_wall([[x0,y0],[x1,y1]])`, how do the place cells respond to the wall? 
* Models: The parameter `params['description']` gets the specific place cell model being used. Try varying this and see what you observe, options include: 
    * `"gaussian"` (default)
    * `"one_hot"`
    * `"gaussian_threshold"`
* Firing rate: Vary the firing rate of the place cells `params["max_fr"]`

Here's some basic code to get you started

In [ ]:
Env = Environment()
Ag = Agent(Env)
PCs = PlaceCells(Ag,params={

    # insert your params here
})

# 10 min simulation 
for i in range(int(10*60/dt)):
    Ag.update(dt=dt)
    PCs.update()

#display the data
PCs.plot_rate_timeseries()
PCs.plot_rate_map(method='neither',spikes=True)

### **2.2 Grid cells** <a name="gridcells"></a>

Other Spatial Representations exist, for example grid cells. The class `GridCells` handles this. 

The default model for grid cells is to sum three cosine wave together 

$$ f_{i}(\vec{x}) = \frac{1}{3} \textrm{fr}_{\textrm{max}} \bigg[ \textrm{cos}\bigg( 2\pi\frac{\vec{x}(t)\cdot \vec{e}_{\theta_{i}}}{\lambda_i} + \phi_{i}\bigg) + \textrm{cos}\bigg( 2\pi\frac{\vec{x}(t)\cdot \vec{e}_{\theta_{i} + \pi/3}}{\lambda_i} + \phi_{i}\bigg) + \textrm{cos}\bigg( 2\pi\frac{\vec{x}(t)\cdot \vec{e}_{\theta_{i}+2\pi/3}}{\lambda_i} + \phi_{i}\bigg)  \bigg]_{+} $$

<center><img src="./figures/display_figures/gridcell_oim.png" width=800></center>

In this model, each grid cell, $i$, has an 
* Orientation, $\theta_i$
* Phase offset, $\phi_i$
* Grid scale, $\lambda_i$

**_📝TASK_**: Initialise, simulate and then visualise some grid cells. To see the available parameters do `GridCells.get_all_default_params()`

Extensions once you've done this: 
* Howdo grid cells respond to adding a wall to the Environment?
* By setting `param['gridscale_distribution']` to `'delta'` and `param['gridscale']` to whatever-value-you-like make a set of grid cells which all have the same gridscale.


In [ ]:
# your code goes here 

In [ ]:
#@title Double click to see solution {display-mode: "form" }
Env = Environment()
Ag = Agent(Env)
GCs = GridCells(Ag,params={'n':5,'max_fr':5})

for i in range(int(10*60/dt)):
    Ag.update(dt=dt)
    GCs.update()

GCs.plot_rate_timeseries()
GCs.plot_rate_map()
GCs.plot_rate_map(method='neither',spikes=True)

### **2.3 Head Direction Cells** <a name="headdirectioncells"></a>

Head direction cells are the brains internal compass. They allow us to know where we're facing and are therefore crucial to the brains ability to path integrate. 

**_📝TASKS_** 
1. Make some `HeadDirectionCells` and explore a 2D environment for 10 minutes
2. Visualise their receptive fields in polar coordinates using `HeadDirectionCells.plot_HDC_receptive_field()`
3. Plot their rate maps using `HeadDirectionCells.plot_rate_map(method='history')`...what do you see


In [ ]:
# your code goes here

In [ ]:
#@title Double click to see solution {display-mode: "form" }
Env = Environment()
Ag = Agent(Env)
HDCs = HeadDirectionCells(Ag,params={'n':10,'max_fr':5})

for i in range(int(10*60/dt)):
    Ag.update(dt=dt)
    HDCs.update()


HDCs.plot_HDC_receptive_field()
HDCs.plot_rate_map(method="history")
HDCs.plot_rate_timeseries(t_end=60)


### **2.4 Boundary Vector Cells** <a name="boundaryvectorcells"></a>

The last cell type we discussed was boundary vector cells. These fire at a fixed angle and distance from the walls.

`RatInABox` contains these as `BoundaryVectorCells()`

**_📝TASK_** 
Remember the Africa `Environment` we built. Let's add some BVCs to that environment and see how they look. Let's start with 5 cells 

In [ ]:
Africa = Environment(params={'boundary':africa_boundary,
                             'dx':0.05 #<--- 5cm resolution (ignore this, it just speeds up the code)
                             })
Ag = Agent(Africa)
BVCs = BoundaryVectorCells(Ag,params={'n':5})

BVCs.plot_rate_map()

---

## **3 Grid cells to place cells** <a name="gc2pc"></a>
### **A model of the place cell formation**

As we discussed in the lecture, it is thought that grid cells provide a primary source of input to place cells. 

Instead of using the `RatInABox` `PlaceCells` class, can we instead sum together `GridCells` in a way which makes place cells??

<center><img src="./figures/display_figures/ec_to_hpc.png" width=400></center>

To do this we'll using the `RatInABox` class called `FeedForwardLayer`. 

<center><img src="./figures/display_figures/feedforwardlayer.png" width=800></center>

`FeedForwardLayer` is quite unique. Unlike `PlaceCells`, `GridCells` etc. who's firing rates are directly determined by ther `Agents` state, this layer takes as input one (or many) other layer, $g_j(t)$ and calculates its firing rate $p_i(t)$$ by summing and "activating" these inputs (like a layer in a deep neural network if you're familiar with those) according to: 

$$
p_{i}(t) = \sigma \big(\sum_j w_{ij} g_{j}(t) \big)
$$

The input layer can be _any other `RatInABox` `Neurons` class_, for example `GridCells`. In which case the new layer calculates an activated linear sum of GridCell inputs. Perhaps this is a good model for hippocampus? Lets see...

In [ ]:
Env = Environment()
Ag = Agent(Env)
GCs = GridCells(Ag,params={'gridscale':np.logspace(np.log10(0.05),np.log10(1),5),
                           'description':'three_shifted_cosines',
                           'orientation_distribution':'delta',
                           'phase_offset_distribution':'delta'})
PCs = FeedForwardLayer(Ag, params={
    'n':1,
    'input_layers':[GCs],
    'activation_params':{'activation':'relu'},
})

fig, ax = GCs.plot_rate_map(chosen_neurons='10')
fig.suptitle("Grid cells: the inputs to our 'place cells'")

fig, ax = PCs.plot_rate_map(chosen_neurons='5')
fig.suptitle("Place cells as a random linear sum of the grid cells. Do these look like place cells? (They shouldn't!)")

Maybe we can do better by thresholding the placec ells near their maximum values .This could be done in the brain 

In [ ]:
ratemaps = PCs.get_state(evaluate_at='all')
max_of_ratemaps = np.max(ratemaps,axis=1)
PCs.biases = -0.5*max_of_ratemaps

PCs.plot_rate_map()

Ok...not great!! Some of them look "place cell-y" but most don't. Perhaps this is because we just chose random weights. It's reasonable to assume the brain wouldn't actually set weights randomly. 

In fact, if you read Solstad et al. (2006) they derived a model for how strong a grid cell should connect to a place cell. It turns out to only depend on the "grid scale" $\lambda$ of the grid cell and the desired size $\sigma_{pc}$ of the place cells. Roughly, their model suggest weights should be set proportional to: 

$$w_{ij} \propto \frac{\exp\bigg(-\frac{4\pi^{2}\sigma_{pc,i}^{2}}{3\lambda_j^{2}}\bigg)}{\lambda_{j}^2}$$

where $\lambda_{j}$ is the gridscale and $\sigma_{pc,i}$ is the size of the place cell. 

In [ ]:
Env = Environment(params={'boundary':[[-0.5,-0.5],[0.5,-0.5],[0.5,0.5],[-0.5,0.5]]})
Ag = Agent(Env,params = {'dt':0.1})
GCs = GridCells(Ag,params={'gridscale':np.logspace(np.log10(0.05),np.log10(1),50),
                           'description':'three_shifted_cosines',
                           'orientation_distribution':'delta',
                           'phase_offset_distribution':'delta'})
PCs = FeedForwardLayer(Ag, params={
    'n':1,
    'input_layers':[GCs],
    'activation_params':{'activation':'relu'},
})

**_📝TASK_** Complete the following code to implement to Solstad model. Use the equation above to help (note there's only 1 place cell) so you can ignore the index i. 

In [ ]:
gridscales = GCs.gridscales
sigma = 0.2
weights = #your code goes here

In [ ]:
#@title Double click to see solution {display-mode: "form" }
gridscales = GCs.gridscales
sigma = 0.2
weights = np.exp(-(4*np.pi**2*sigma**2)/(3*gridscales**2))/gridscales**2

In [ ]:
#Here we set the weights by accessing the weight matrix. 
PCs.inputs['GridCells']['w'] = weights

#Threshold at 50% the max 
ratemaps = PCs.get_state(evaluate_at='all')
max_of_ratemaps = np.max(ratemaps,axis=1)
PCs.biases = -0.5*max_of_ratemaps

fig, ax = PCs.plot_rate_map()
fig.suptitle("A place cell, calculated as a weighted linear sum of the grid cells.\nDoes it look like a place cell? (It should!)")

---
## **4.0 Advanced exercises** <a name="advanced"></a>

In any remaining time here's a list of task you might like to try. Pick one you think looks fun and give it a go (the content above should be enough to get you there):

**_📝TASK A_** Make a rectangular environment with two square holes so that it forms a figure-of-eight. Then, add an agent to the environment and by designing an appropriate `drift_velocity` make it move around the arms of the figure-of-eight.

**_📝TASK B_** Initialise a population of `GridCells` and a population of `HeadDirectionCells`. Using a `FeedForwardLayer` to create neurons which "mix" these two cell types. How might this be related to the role of the entorhinal cortex in path integration (i.e. dead reckoning). 

**_📝TASK C_** Add one or many objects to the `Environment` using `Env.add_object(position=[x,y])` (I didn't tell you about this functionality). Then initialise a population of `ObjectVectorCells` and plot their receptive field. What do you observe. 

**_📝TASK D_** (Very hard) Generate data from a population of 10 place cells as an `Agent` randomly explores a 2D Environment. Extract the firing rate data from the `PlaceCells.history` data frame. Extract the position data from the `Agent.history` data frame. Trai na regression decoder to _predict_ the position from the firing rate. You may like to check out [scikit-learns linear regression model](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) to do this. We will do something like this,and more, next Thursday. 

---

## **Congrats!** 
Well done. You made it to the end of the tutorial! 

### **Re-use**
Feel free you to adapt and use this tutorial for your own teaching needs! 

### **About the author: Tom George**
* Feel free to get in touch at tom.george.20@ucl.ac.uk 
* Links: [Twitter](https://twitter.com/TomNotGeorge), [Github](https://github.com/TomGeorge1234), [Google Scholar](https://scholar.google.com/citations?user=AG49j3MAAAAJ&hl=en)

In [ ]:
africa_boundary = np.array([
[-0.83935318,  0.30607098],
[-0.83532515,  0.18512068],
[-0.84741901,  0.06820623],
[-0.75063883, -0.06080795],
[-0.61756729, -0.16160638],
[-0.48852379, -0.1414467 ],
[-0.43207275, -0.13338087],
[-0.29093539, -0.10919315],
[-0.27076592, -0.16160638],
[-0.17398574, -0.16160638],
[-0.16591991, -0.25836702],
[-0.20221126, -0.31480827],
[-0.09736524, -0.41559693],
[-0.0651117 , -0.58493048],
[-0.12560053, -0.70991663],
[-0.07720555, -0.82682718],
[-0.01671671, -1.02034844],
[ 0.05183795, -1.1251749 ],
[ 0.03536411, -1.19568492],
[ 0.08340712, -1.21970643],
[ 0.18979787, -1.19225328],
[ 0.26873546, -1.18195835],
[ 0.40944265, -1.0344172 ],
[ 0.40944265, -0.95893081],
[ 0.47464389, -0.91776086],
[ 0.47464389, -0.7907998 ],
[ 0.62222414, -0.67071181],
[ 0.57760299, -0.40650454],
[ 0.69428866, -0.25210011],
[ 0.76293132, -0.21779345],
[ 0.88991193, -0.00505106],
[ 0.87618536,  0.03955053],
[ 0.71144688, -0.00848271],
[ 0.67026715,  0.0292556 ],
[ 0.59132956,  0.13905648],
[ 0.56387641,  0.20768056],
[ 0.52955997,  0.22483683],
[ 0.53642326,  0.30375487],
[ 0.40944265,  0.51992107],
[ 0.46434895,  0.47531556],
[ 0.47464389,  0.52678436],
[ 0.44719073,  0.57138987],
[ 0.34423163,  0.57482151],
[ 0.33050506,  0.5542336 ],
[ 0.13488179,  0.61599635],
[ 0.08683877,  0.59884009],
[ 0.09027041,  0.54737129],
[ 0.05938562,  0.53021503],
[-0.02641526,  0.58511449],
[-0.15682752,  0.65030791],
[-0.12937436,  0.68118879],
[-0.14996423,  0.75324452],
[-0.21517524,  0.74295057],
[-0.42108367,  0.72579528],
[-0.46570482,  0.68118879],
[-0.54120099,  0.68462044],
[-0.56866392,  0.70863901],
[-0.59611707,  0.64687724],
[-0.66474996,  0.61599635],
[-0.67162302,  0.54394063],
[-0.7402559 ,  0.46502259],
[-0.79174034,  0.40669148],
[-0.82948843,  0.32777344]])

accra = np.array([-0.4,-0.1])
kigali = np.array([0.2,0.0])